Загружаем необходимые библиотеки

In [1]:
from openpyxl import load_workbook
import pandas as pd
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC

С помощью библиотеки Pandas непосредственно невозможно получить информацию о цвете текста в файле Excel. Выполним разделение на классы перед выполнением программы

In [2]:
book = load_workbook('datasets/V1RS41at_02.05.2014ТЗЧ.xlsx')
sheet = book.active
classes = []
ignore_columns = []

for column in range(1, sheet.max_column):
    if sheet[1][column].fill.fgColor.rgb == 'FFFFFF00':
        ignore_columns.append(column)

for row in range(2,  sheet.max_row):
    red_cell_in_row = False
    for column in range(1, sheet.max_column):
        if column not in ignore_columns:
            cell = sheet[row][column]
            row_value = row - 1
            while cell.value is None:
                cell = sheet[row_value][column]
                row_value -= 1
            if cell.font.color.value == 'FFFF0000':
                red_cell_in_row = True
                break

    if red_cell_in_row:
        if classes[-1] == 0:
            classes.append(2)
        elif classes[-1] == 1:
            classes.append(1)
        elif classes[-3:] == [2, 2, 2]:
            classes = classes[:-3] + [1, 1, 1, 1]
        elif classes[-1] == 2:
            classes.append(2)
    else:
        classes.append(0)

print(classes)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 2, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 

Считываем данные

In [3]:
data = pd.read_excel('datasets/V1RS41at_02.05.2014ТЗЧ.xlsx')
data

,Unnamed: 0,"Ubs,V","Ibs,A","Isun,A","RKPT,b","Sst0x,b","Sst1x,b","Ipt0,A","Ipt1,A","Ipt2,A",...,"SPrm2,V","VP,d","ShKom,d","Kmnd,d","God,h","Mes,h","Den,h","Chas,h","Min,h",Sec
0,2014-05-02 00:00:31,14.63,0.27,0.13,1000000.0,NaN,0.0,0.0,0.09,0.09,...,0.98,0.0,29.0,430.0,14,5,2,0,0,31
1,2014-05-02 00:01:31,14.76,NaN,NaN,1000000.0,110.0,0.0,NaN,NaN,0.07,...,0.96,0.0,29.0,430.0,14,5,2,0,1,NaN
2,2014-05-02 00:02:31,13.64,0.24,0.72,1000000.0,110.0,0.0,0.0,0.09,0.09,...,0.98,0.0,29.0,NaN,NaN,NaN,2,0,2,31
3,2014-05-02 00:03:31,14.69,0.21,0.56,1000000.0,110.0,0.0,0.0,0.07,0.09,...,0.98,0.0,29.0,430.0,14,5,2,0,3,31
4,2014-05-02 00:04:31,14.76,0.21,0.83,1000000.0,110.0,0.0,0.0,0.07,0.07,...,0.96,0.0,29.0,430.0,14,5,2,0,4,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1433,2014-05-02 23:55:10,14.56,0.21,0.51,1000000.0,110.0,0.0,0.0,0.09,0.07,...,1.06,0.0,83.0,2771.0,14,5,2,23,55,10
1434,2014-05-02 23:56:10,14.56,0.21,0.64,1000000.0,110.0,0.0,0.0,0.09,0.09,...,1.02,0.0,83.0,2771.0,14,5,2,23,56,10
1435,2014-05-02 23:57:10,14.56,0.27,0.69,1000000.0,110.0,0.0,0.0,NaN,0.07,...,0.99,0.0,83.0,2771.0,14,5,2,23,57,10
1436,2014-05-02 23:58:11,14.56,0.19,0.43,1000000.0,NaN,0.0,0.0,NaN,0.13,...,NaN,0.0,83.0,NaN,14,5,2,NaN,58,10


Удаляем необходимые столбцы

In [4]:
signs_to_remove = ['RKPT,b','Sst0x,b','Sst1x,b','Ipt0,A', 'Mag0F', 'Mag10', 'Mag11', 'Mag12', 'Mag13',
                   'Mag14','Mag15','Mag16','Isp1,A','Isp2,A','Isp3,A','Isp4,A','Isp5,A','Isp6,A','Isp7,A',
                   'Isp8,A','Mag07','Mag08','Mag09','Mag0A','Mag0B','Mag0C','Mag0D','Mag0E','Mag03','Mag04',
                   'Mag05','Mag06','TDS11,C','TDS12,C','TDS14,C','TDS15,C','TDS17,C','TDS18,C','TDS20,C',
                   'TDS21,C','TDS23,C','iTMO,s','iTMD,s','U-chCAN', 'SSoTR,b','STDS12,b','STDS34,b','res0',
                   'U-MAG,o','U-KPT1,o','U-KPT2,o','U-InfGBV','U-GBV,o','U-InfNAP','TmUpr,b','RKwit,b','Rmorz,b',
                   'IntK_TM', 'RPrd1,b','RPrm1,b','RPrd2,b','RPrm2,b','FPrd1,V','RPrd1,V','FPrd2,V','RPrd2,V','SPrm1,V',
                   'SPrm2,V','VP,d','ShKom,d','Kmnd,d','God,h','Mes,h','Den,h','Chas,h', 'Min,h', 'Sec']
cleared_data = data.copy()
cleared_data.drop(data.columns[[0]], axis=1, inplace=True)
for sign in signs_to_remove:
    cleared_data.drop(columns=[sign], axis= 1, inplace=True)
cleared_data

,"Ubs,V","Ibs,A","Isun,A","Ipt1,A","Ipt2,A","Ipt3,A","Ipt4,A","Ipt5,A","Ipt6,A","Ipt7,A",...,"TDS6,C","TDS7,C","TDS8,C","TDS9,C","TKpt,C","TGbv,C","TNap,C","TPrd2,C","TPrd1,C","TDS24,C"
0,14.63,0.27,0.13,0.09,0.09,0.0,0.0,0.0,0.0,0.0,...,15.0,15.0,15.0,15.0,24.0,16.0,16.0,16.0,16.0,19.0
1,14.76,NaN,NaN,NaN,0.07,0.0,0.0,0.0,0.0,0.0,...,15.0,NaN,NaN,15.0,24.0,16.0,16.0,NaN,16.0,19.0
2,13.64,0.24,0.72,0.09,0.09,0.0,0.0,0.0,0.0,0.0,...,15.0,16.0,15.0,15.0,24.0,16.0,16.0,16.0,16.0,19.0
3,14.69,0.21,0.56,0.07,0.09,0.0,0.0,0.0,0.0,0.0,...,15.0,16.0,16.0,15.0,24.0,16.0,16.0,16.0,16.0,19.0
4,14.76,0.21,0.83,0.07,0.07,0.0,0.0,0.0,0.0,0.0,...,15.0,16.0,16.0,15.0,24.0,16.0,16.0,17.0,16.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1433,14.56,0.21,0.51,0.09,0.07,0.0,0.0,0.0,0.0,0.0,...,15.0,15.0,15.0,15.0,25.0,18.0,17.0,17.0,17.0,21.0
1434,14.56,0.21,0.64,0.09,0.09,0.0,0.0,0.0,0.0,0.0,...,15.0,15.0,15.0,15.0,25.0,17.0,17.0,17.0,17.0,21.0
1435,14.56,0.27,0.69,NaN,0.07,0.0,0.0,0.0,0.0,0.0,...,15.0,15.0,15.0,15.0,25.0,17.0,17.0,17.0,17.0,21.0
1436,14.56,0.19,0.43,NaN,0.13,0.0,NaN,0.0,0.0,0.0,...,15.0,15.0,15.0,15.0,25.0,17.0,NaN,NaN,17.0,21.0


Заполняем пустые значения

In [5]:
data_filled = cleared_data.copy()
data_filled = data_filled.fillna(method='ffill')
data_filled

,"Ubs,V","Ibs,A","Isun,A","Ipt1,A","Ipt2,A","Ipt3,A","Ipt4,A","Ipt5,A","Ipt6,A","Ipt7,A",...,"TDS6,C","TDS7,C","TDS8,C","TDS9,C","TKpt,C","TGbv,C","TNap,C","TPrd2,C","TPrd1,C","TDS24,C"
0,14.63,0.27,0.13,0.09,0.09,0.0,0.0,0.0,0.0,0.0,...,15.0,15.0,15.0,15.0,24.0,16.0,16.0,16.0,16.0,19.0
1,14.76,0.27,0.13,0.09,0.07,0.0,0.0,0.0,0.0,0.0,...,15.0,15.0,15.0,15.0,24.0,16.0,16.0,16.0,16.0,19.0
2,13.64,0.24,0.72,0.09,0.09,0.0,0.0,0.0,0.0,0.0,...,15.0,16.0,15.0,15.0,24.0,16.0,16.0,16.0,16.0,19.0
3,14.69,0.21,0.56,0.07,0.09,0.0,0.0,0.0,0.0,0.0,...,15.0,16.0,16.0,15.0,24.0,16.0,16.0,16.0,16.0,19.0
4,14.76,0.21,0.83,0.07,0.07,0.0,0.0,0.0,0.0,0.0,...,15.0,16.0,16.0,15.0,24.0,16.0,16.0,17.0,16.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1433,14.56,0.21,0.51,0.09,0.07,0.0,0.0,0.0,0.0,0.0,...,15.0,15.0,15.0,15.0,25.0,18.0,17.0,17.0,17.0,21.0
1434,14.56,0.21,0.64,0.09,0.09,0.0,0.0,0.0,0.0,0.0,...,15.0,15.0,15.0,15.0,25.0,17.0,17.0,17.0,17.0,21.0
1435,14.56,0.27,0.69,0.09,0.07,0.0,0.0,0.0,0.0,0.0,...,15.0,15.0,15.0,15.0,25.0,17.0,17.0,17.0,17.0,21.0
1436,14.56,0.19,0.43,0.09,0.13,0.0,0.0,0.0,0.0,0.0,...,15.0,15.0,15.0,15.0,25.0,17.0,17.0,17.0,17.0,21.0


Выполним преобразование данных

In [6]:
# Преобразовываем исходный набор
scaler = MinMaxScaler(feature_range=(0, 1))
rescaled_data = pd.DataFrame(scaler.fit_transform(data_filled))
rescaled_data

,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
0,0.887993,0.035556,0.019345,0.027778,0.017176,0.0,0.0,0.0,0.0,0.0,...,0.711443,0.687500,0.959732,0.711443,0.713615,0.716418,0.911392,0.716418,0.842105,0.727723
1,0.899642,0.035556,0.019345,0.027778,0.013359,0.0,0.0,0.0,0.0,0.0,...,0.711443,0.687500,0.959732,0.711443,0.713615,0.716418,0.911392,0.716418,0.842105,0.727723
2,0.799283,0.028889,0.107143,0.027778,0.017176,0.0,0.0,0.0,0.0,0.0,...,0.711443,0.692308,0.959732,0.711443,0.713615,0.716418,0.911392,0.716418,0.842105,0.727723
3,0.893369,0.022222,0.083333,0.021605,0.017176,0.0,0.0,0.0,0.0,0.0,...,0.711443,0.692308,0.966443,0.711443,0.713615,0.716418,0.911392,0.716418,0.842105,0.727723
4,0.899642,0.022222,0.123512,0.021605,0.013359,0.0,0.0,0.0,0.0,0.0,...,0.711443,0.692308,0.966443,0.711443,0.713615,0.716418,0.911392,0.721393,0.842105,0.727723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1433,0.881720,0.022222,0.075893,0.027778,0.013359,0.0,0.0,0.0,0.0,0.0,...,0.711443,0.687500,0.959732,0.711443,0.718310,0.726368,0.917722,0.721393,0.847953,0.737624
1434,0.881720,0.022222,0.095238,0.027778,0.017176,0.0,0.0,0.0,0.0,0.0,...,0.711443,0.687500,0.959732,0.711443,0.718310,0.721393,0.917722,0.721393,0.847953,0.737624
1435,0.881720,0.035556,0.102679,0.027778,0.013359,0.0,0.0,0.0,0.0,0.0,...,0.711443,0.687500,0.959732,0.711443,0.718310,0.721393,0.917722,0.721393,0.847953,0.737624
1436,0.881720,0.017778,0.063988,0.027778,0.024809,0.0,0.0,0.0,0.0,0.0,...,0.711443,0.687500,0.959732,0.711443,0.718310,0.721393,0.917722,0.721393,0.847953,0.737624


Присоединяем столб классов

In [7]:
rescaled_data_with_classes = rescaled_data.copy()
data_with_classes = data_filled.copy()

rescaled_data_with_classes.insert(loc = rescaled_data_with_classes.shape[1], column = 'Class', value = classes)
data_with_classes.insert(loc = data_with_classes.shape[1], column = 'Class', value = classes)
data_with_classes

,"Ubs,V","Ibs,A","Isun,A","Ipt1,A","Ipt2,A","Ipt3,A","Ipt4,A","Ipt5,A","Ipt6,A","Ipt7,A",...,"TDS7,C","TDS8,C","TDS9,C","TKpt,C","TGbv,C","TNap,C","TPrd2,C","TPrd1,C","TDS24,C",Class
0,14.63,0.27,0.13,0.09,0.09,0.0,0.0,0.0,0.0,0.0,...,15.0,15.0,15.0,24.0,16.0,16.0,16.0,16.0,19.0,0
1,14.76,0.27,0.13,0.09,0.07,0.0,0.0,0.0,0.0,0.0,...,15.0,15.0,15.0,24.0,16.0,16.0,16.0,16.0,19.0,0
2,13.64,0.24,0.72,0.09,0.09,0.0,0.0,0.0,0.0,0.0,...,16.0,15.0,15.0,24.0,16.0,16.0,16.0,16.0,19.0,0
3,14.69,0.21,0.56,0.07,0.09,0.0,0.0,0.0,0.0,0.0,...,16.0,16.0,15.0,24.0,16.0,16.0,16.0,16.0,19.0,0
4,14.76,0.21,0.83,0.07,0.07,0.0,0.0,0.0,0.0,0.0,...,16.0,16.0,15.0,24.0,16.0,16.0,17.0,16.0,19.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1433,14.56,0.21,0.51,0.09,0.07,0.0,0.0,0.0,0.0,0.0,...,15.0,15.0,15.0,25.0,18.0,17.0,17.0,17.0,21.0,0
1434,14.56,0.21,0.64,0.09,0.09,0.0,0.0,0.0,0.0,0.0,...,15.0,15.0,15.0,25.0,17.0,17.0,17.0,17.0,21.0,0
1435,14.56,0.27,0.69,0.09,0.07,0.0,0.0,0.0,0.0,0.0,...,15.0,15.0,15.0,25.0,17.0,17.0,17.0,17.0,21.0,0
1436,14.56,0.19,0.43,0.09,0.13,0.0,0.0,0.0,0.0,0.0,...,15.0,15.0,15.0,25.0,17.0,17.0,17.0,17.0,21.0,2


Проверим, заполнились ли пустые значение

In [8]:
data_with_classes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1438 entries, 0 to 1437
Data columns (total 50 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Ubs,V    1438 non-null   float64
 1   Ibs,A    1438 non-null   float64
 2   Isun,A   1438 non-null   float64
 3   Ipt1,A   1438 non-null   float64
 4   Ipt2,A   1438 non-null   float64
 5   Ipt3,A   1438 non-null   float64
 6   Ipt4,A   1438 non-null   float64
 7   Ipt5,A   1438 non-null   float64
 8   Ipt6,A   1438 non-null   float64
 9   Ipt7,A   1438 non-null   float64
 10  Ipt10,A  1438 non-null   float64
 11  Ipt11,A  1438 non-null   float64
 12  Ipt12,A  1438 non-null   float64
 13  Ipt13,A  1438 non-null   float64
 14  Ipt14,A  1438 non-null   float64
 15  Ipt15,A  1438 non-null   float64
 16  Ipt16,A  1438 non-null   float64
 17  Ipt17,A  1438 non-null   float64
 18  TR1,C    1438 non-null   float64
 19  TR2,C    1438 non-null   float64
 20  TR3,C    1438 non-null   float64
 21  TR4,C    1438 

Проверим, сколько образцов относятся к каждому классу

In [9]:
class_counts = data_with_classes['Class'].value_counts()
class_counts

Class
0    992
1    305
2    141
Name: count, dtype: int64

Разделим набор на обучащую и тестовую выборки

In [10]:
test_size = 0.2
seed = 7

data_X = data_with_classes.iloc[:, :-1]
data_Y = data_with_classes['Class']

data_X_train, data_X_test, data_Y_train, data_Y_test = train_test_split(data_X, data_Y, test_size=test_size, random_state=seed)

rescaled_data_X = rescaled_data_with_classes.iloc[:, :-1]
rescaled_data_Y = rescaled_data_with_classes['Class']

rescaled_data_X_train, rescaled_data_X_test, rescaled_data_Y_train, rescaled_data_Y_test = train_test_split(rescaled_data_X, rescaled_data_Y, test_size=test_size, random_state=seed)


datasets = [#{'X_train': data_X_train, 'X_test': data_X_test, 'Y_train': data_Y_train, 'Y_test': data_Y_test, 'name': 'original_data'},
            {'X_train': rescaled_data_X_train, 'X_test': rescaled_data_X_test, 'Y_train': rescaled_data_Y_train, 'Y_test': rescaled_data_Y_test, 'name': 'original_rescaled_data'}]

Определим гиперпараметры для SVC

In [11]:
param_grid = {'C': [0.1, 1, 10, 100],
              'kernel': ['linear', 'poly']}

Реализация SVC

In [12]:
for dataset in datasets:
    print(f'Analyse for {dataset.get("name")}')
    svc = SVC()
    grid_search = GridSearchCV(estimator=svc, param_grid=param_grid)
    grid_search.fit(dataset.get('X_train'), dataset.get('Y_train'))
    best_params_for_data = grid_search.best_params_
    print(f"Best hyper parameters for SVC: {best_params_for_data}")

    svc_model_train_data = SVC(C=best_params_for_data.get('C'),
                               kernel=best_params_for_data.get('kernel'))
    svc_model_train_data.fit(dataset.get('X_train'), dataset.get('Y_train'))
    train_Y_pred = svc_model_train_data.predict(dataset.get('X_train'))
    print('Classification report for train data')
    print(classification_report(dataset.get('Y_train'), train_Y_pred))

    svc_model_test_data = SVC(C=best_params_for_data.get('C'),
                              kernel=best_params_for_data.get('kernel'))
    svc_model_test_data.fit(dataset.get('X_train'), dataset.get('Y_train'))
    test_Y_pred = svc_model_train_data.predict(dataset.get('X_test'))
    print('Classification report for test data')
    print(classification_report(dataset.get('Y_test'), test_Y_pred))

Analyse for original_rescaled_data
Best hyper parameters for SVC: {'C': 10, 'kernel': 'poly'}
Classification report for train data
              precision    recall  f1-score   support

           0       0.94      1.00      0.97       804
           1       0.98      0.92      0.95       230
           2       0.93      0.66      0.77       116

    accuracy                           0.95      1150
   macro avg       0.95      0.86      0.90      1150
weighted avg       0.95      0.95      0.95      1150

Classification report for test data
              precision    recall  f1-score   support

           0       0.91      0.99      0.95       188
           1       0.92      0.79      0.85        75
           2       0.50      0.36      0.42        25

    accuracy                           0.89       288
   macro avg       0.78      0.71      0.74       288
weighted avg       0.88      0.89      0.88       288



Определим гиперпараметры для бэггинга

In [13]:
param_grid = {'n_estimators': [500],
              'max_samples': [0.5, 0.75, 1.0],
              'estimator__C': [0.1, 1, 10, 100],
              'estimator__kernel': ['poly']}

Реализация Бэггинга на основе SVC

In [14]:
for dataset in datasets:
    print(f'Analyse for {dataset.get("name")}')
    svc_model = SVC()
    bagging_model = BaggingClassifier(svc_model)
    grid_search = GridSearchCV(bagging_model, param_grid)
    grid_search.fit(dataset.get('X_train'), dataset.get('Y_train'))

    best_params_for_data = grid_search.best_params_
    print(f"Best hyper parameters for bagging: {best_params_for_data}")

    svc_model_train_data = SVC(C=best_params_for_data.get('estimator__C'),
                               kernel=best_params_for_data.get('estimator__kernel'))
    bagging_model_train_data = BaggingClassifier(svc_model_train_data,
                                                 n_estimators=best_params_for_data.get('n_estimators'),
                                                 max_samples=best_params_for_data.get('max_samples'))

    bagging_model_train_data.fit(dataset.get('X_train'), dataset.get('Y_train'))
    train_Y_pred = bagging_model_train_data.predict(dataset.get('X_train'))
    print('Classification report for train data')
    print(classification_report(dataset.get('Y_train'), train_Y_pred))

    svc_model_test_data = SVC(C=best_params_for_data.get('estimator__C'),
                              kernel=best_params_for_data.get('estimator__kernel'))
    bagging_model_test_data = BaggingClassifier(svc_model_test_data,
                                                n_estimators=best_params_for_data.get('n_estimators'),
                                                max_samples=best_params_for_data.get('max_samples'))
    bagging_model_test_data.fit(dataset.get('X_train'), dataset.get('Y_train'))
    test_Y_pred = bagging_model_test_data.predict(dataset.get('X_test'))
    print('Classification report for test data')
    print(classification_report(dataset.get('Y_test'), test_Y_pred))

Analyse for original_rescaled_data
Best hyper parameters for bagging: {'estimator__C': 10, 'estimator__kernel': 'poly', 'max_samples': 1.0, 'n_estimators': 500}
Classification report for train data
              precision    recall  f1-score   support

           0       0.95      1.00      0.97       804
           1       0.98      0.93      0.96       230
           2       0.94      0.68      0.79       116

    accuracy                           0.95      1150
   macro avg       0.96      0.87      0.91      1150
weighted avg       0.95      0.95      0.95      1150

Classification report for test data
              precision    recall  f1-score   support

           0       0.90      1.00      0.95       188
           1       0.94      0.80      0.86        75
           2       0.56      0.36      0.44        25

    accuracy                           0.89       288
   macro avg       0.80      0.72      0.75       288
weighted avg       0.88      0.89      0.88       288



Выводы: На преобразованном наборе данных, такие же или лучшие показатели, что на обучающем наборе данных, что на тестовом показывает бэггинг. Обели модели показывают плохие оценки для второго класса на тестовом наборе данных. Остальные оценки лежат на уровне 80%. В итоге можно сказать, что бэггинг на основе SVC с параметрами {'estimator__C': 10, 'estimator__kernel': 'poly', 'max_samples': 1.0, 'n_estimators': 500} лучше, чем SVC с параметрами {'C': 10, 'kernel': 'poly'} для данного набора данных.